# Openrouteservice

In [2]:
import os
import xlrd
import folium
import pandas as pd
from pandas import DataFrame
import openrouteservice as ors

In [14]:
# Read the files
rent = pd.read_csv("../rent_data/renew(final).csv", index_col=0)
train = pd.read_excel("../data/raw/train_station_20to21.xlsx", index_col=0)
distance_duration = pd.read_csv("../distances_and_durations/train_old/train_distance_duration(final).csv", index_col=0).reset_index(drop=True)

# API key
key = os.environ.get('key')
client = ors.Client(key= key)

In [15]:
def coordinates(rent, train, distance_duration, count):
    """
    function: Draw a map based on the data obtained by comparing the house with the station
    para: map
    """
    map = folium.Map(location=[-37.81361, 144.96306], tiles='cartodbpositron', zoom_start=13)
    for i in range(count):
        get_data = distance_duration.loc[i]
        coordinates = [[float(rent[rent['listingSlug'] ==  get_data['rent_address']]['longitude']), float(rent[rent['listingSlug'] ==  get_data['rent_address']]['latitude'])], \
            [float(train[train['Stop_name'] == get_data['stop_name']]['Stop_long']), float(train[train['Stop_name'] == get_data['stop_name']]['Stop_lat'])]]
        #When drawing, first latitude and then longitude, when calculating distance, first longitude and then latitude
        coordinates2 = [[float(rent[rent['listingSlug'] ==  get_data['rent_address']]['latitude']), float(rent[rent['listingSlug'] ==  get_data['rent_address']]['longitude'])], \
            [float(train[train['Stop_name'] == get_data['stop_name']]['Stop_lat']), float(train[train['Stop_name'] == get_data['stop_name']]['Stop_long'])]]
        # calculate route
        route = client.directions(
            coordinates=coordinates,
            profile='driving-car',
            format='geojson',
            validate=False,
            )
        folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]).add_to(map)
        # draw points
        folium.Marker(
        location=coordinates2[0],
        popup="Rent",
        icon=folium.Icon(color="green"),
        ).add_to(map)
        folium.Marker(
        location=coordinates2[1],
        popup="train",
        icon=folium.Icon(color="red", icon="info-sign"),
        ).add_to(map)
    return map

In [16]:
coordinates(rent, train, distance_duration,3)

In [18]:
coordinates(rent, train, distance_duration, 3).save('../plots/route_map.html')